In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

In [2]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

train_df['Word'] = train_df['Word'].apply(lambda x: x.lower())
test_df['Word'] = test_df['Word'].apply(lambda x: x.lower())
train_df = train_df.drop_duplicates();

train_df['Word'] = (train_df['Word']).str.replace('[^а-я\-]+', '')
test_df['Word'] = (test_df['Word']).str.replace('[^а-я\-]+', '')

In [27]:
vectorizer = TfidfVectorizer(analyzer='char',\
                                    ngram_range=(1,4))

combined = (train_df['Word']).append(test_df['Word'], ignore_index=True)
train_vect = vectorizer.fit_transform(combined)

In [28]:
X=train_vect[:89907]
X_test = train_vect[89907:]

In [29]:
log_params = {'C': [0.001, 0.01, 0.1, 0.5, 1, 3, 5, 10, 100, 500, 1000]}

In [30]:
clf = LogisticRegression(penalty = 'l2', fit_intercept=True, class_weight='balanced', random_state=0)
log_grid = GridSearchCV(clf, log_params, cv =5, n_jobs=-1)

In [32]:
%%time
log_grid.fit(X, train_df['Label'])

Wall time: 1min 28s


GridSearchCV(cv=5, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=0,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'C': [0.001, 0.01, 0.1, 0.5, 1, 3, 5, 10, 100, 500, 1000]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [33]:
predictions = log_grid.best_estimator_.predict_proba(X_test)

In [34]:
submission = pd.DataFrame(predictions[:,1])

submission.index.name = 'Id'
submission.columns = ['Prediction']

submission.to_csv('results.csv', header=True)